In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-09-07 16:25:38


In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2012,2017)
MaxOpenProcesses = 15
Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening'
ReceivedDir  = Project_DIR + '\\2.Received_Data'
Project_GDB  = Project_DIR + '\\5.NM_NS_GDB.gdb'
Python_Dir   = Project_DIR + '\\6.Python_Notebook'

Src_Loc  = ReceivedDir + '\\crash_asof_11jun2018.csv'
Src_Unit = ReceivedDir + '\\vehicle_asof_11jun2018.csv'
Src_Occ  = ReceivedDir + '\\occupant_asof_11jun2018.csv'

HPMS105 = {year:ReceivedDir + '\\v105\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}
HPMS10  = {year:ReceivedDir + '\\v10\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}

Routes  = {year:Project_GDB + '\\Routes_{}'.format(year) for year in Years}
AttTabs = {year:Project_GDB + '\\SegAtt_{}'.format(year) for year in Years}
Segs    = {year:Project_GDB + '\\Seg_{}'.format(year) for year in Years}
CrashData = {year:Project_GDB + '\\Crash_{}'.format(year) for year in range(2012,2018)}
Segs[2017] = Segs[2016]


UTM13N = arcpy.SpatialReference(26913)
NAD83_NM = arcpy.SpatialReference(6529)

RouteID = "ROUTE_ID"
BEG_STA = 'BEG_POINT'
END_STA = 'END_POINT'
Fields = ['F_SYSTEM','THROUGH_LANES','FACILITY_TYPE','AADT','MEDIAN_TYPE','SPEED_LIMIT','LANE_WIDTH']

BridgeUnder = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeUnder'
BridgeLRS   = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeLRS'
Signals     = ReceivedDir + '\\v10\\jg_request.gdb\\SignalizedIntersections'

2018-09-07 16:25:38


In [15]:
# Read Route Data
print(strftime("%Y-%m-%d %H:%M:%S"))
Route_Dict = {year:hsmpy3.common.FCtoDF(Routes[year],True) for year in [2016]}
Route_Dict[2017] = Route_Dict[2016]
Route_Dict[2012] = Route_Dict[2016]
Route_Dict[2013] = Route_Dict[2016]
Route_Dict[2014] = Route_Dict[2016]
Route_Dict[2015] = Route_Dict[2016]
#Route_Dict = {year:hsmpy3.common.AttributeTabletoDF(Routes[year],True) for year in [2016]}

2018-08-29 08:37:10


In [3]:
# Read Crash Data
print(strftime("%Y-%m-%d %H:%M:%S"))
Crash_Dict = {year:hsmpy3.common.FCtoDF(CrashData[year],True) for year in range(2012,2018)}

2018-08-29 12:27:05


In [5]:
# Read Source Crash Data
print(strftime("%Y-%m-%d %H:%M:%S"))
SrcLoc_DF = pd.read_csv(Src_Loc)
SrcLoc_DF2017 = pd.read_csv(ReceivedDir + '\\Ped_2017_KAB\\crashkabped.csv')
SrcLoc_DF = pd.concat([SrcLoc_DF,SrcLoc_DF2017[SrcLoc_DF2017.year==2017]])

2018-08-29 08:27:23


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(strftime("%Y-%m-%d %H:%M:%S"))
p=hsmpy3.nm.CON_AddRouteDirection(HSMPY_PATH,Python_Dir,Routes[2016])

2018-08-28 09:27:16


In [4]:
# Spatial Join: Crash Data and Routes
print(strftime("%Y-%m-%d %H:%M:%S"))
SPJ_Dict = {}
for year in range(2012,2018):
    SPJ = hsmpy3.common.CreateOutPath(Project_GDB + '\\Crash_SPJ',str(year),'')
    arcpy.analysis.SpatialJoin(
        target_features = CrashData[year],
        join_features = Segs[year], 
        out_feature_class = SPJ, 
        join_operation = 'JOIN_ONE_TO_MANY', 
        join_type = 'KEEP_ALL', 
        field_mapping = '#', 
        match_option = 'WITHIN_A_DISTANCE', 
        search_radius = '200 Feet', 
        distance_field_name = ''
    )
    SPJ_Dict.update({year:hsmpy3.common.AttributeTabletoDF(SPJ,False)})
    print(year,SPJ_Dict[year].shape)
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-29 08:15:55
2012 (15400, 31)
2013 (16087, 31)
2014 (16775, 31)
2015 (19767, 31)
2016 (19488, 31)
2017 (1442, 30)
2018-08-29 08:27:23


In [ ]:
# Crash Assignment
print(strftime("%Y-%m-%d %H:%M:%S"))
def ExtractRoute(Route_DF,GIS_RID,GIS_DIR):
    r = str(GIS_RID).split(' ')
    if len(r)>1:
        if r[1][-1] in ['E','N','W','S']:
            rid = '-'.join([r[0],r[1][:-1],'P'])
            GIS_DIR = r[1][-1]
        else:
            rid = '-'.join([r[0],r[1],'P'])
    else:
        rid = GIS_RID
    if rid in list(Route_DF.ROUTE_ID):
        if OppositeDirID(rid) in list(Route_DF.ROUTE_ID):
            if GIS_DIR == list(Route_DF[Route_DF.ROUTE_ID==OppositeDirID(rid)]['Direction'])[0]:
                rid = OppositeDirID(rid)
        return(rid)
    else:
        return('')
def GetDistClass(dist):
    if dist<80:
        return(1)
    if dist<150:
        return(2)
    return(3)
def OppositeDirID(InpID):
    out = InpID[:-2]
    ext = InpID[-1]
    if ext == 'P':
        return('{}-M'.format(out))
    if ext == 'M':
        return('{}-P'.format(out))
for year in range(2012,2018):
    f = FloatProgress(min=0, max=Crash_Dict[year].shape[0],description = str(year))
    display(f)
    Crash_Dict[year]['RID'] = ''
    Crash_Dict[year]['MP'] = 0.0
    Crash_Dict[year]['Offset'] = 0.0
    for i,r in Crash_Dict[year].iterrows():
        ext_rid = ''
        spjdf = SPJ_Dict[year][(SPJ_Dict[year].UCRnum==r.UCRnum) & SPJ_Dict[year].Join_Count>0]
        if len(list(spjdf.ROUTE_ID))>0:
            srcdf = SrcLoc_DF[(SrcLoc_DF.UCRnumber==int(r.UCRnum)) & (SrcLoc_DF.year==r.Date.year)]
            gis_rid = list(srcdf.GIS_Route)[0]
            gis_dir = list(srcdf.CrashDirection)[0]
            ext_rid = ExtractRoute(Route_Dict[year],gis_rid,gis_dir)
            if not ext_rid in list(spjdf.ROUTE_ID):
                spjdf['Dist_To_Crash'] = [Route_Dict[year][Route_Dict[year]['ROUTE_ID']==spj_rid]['Shape'].item().projectAs(NAD83_NM).distanceTo(r['Shape']) 
                                 for spj_rid in list(spjdf.ROUTE_ID)]
                spjdf['Dist_Class'] = [GetDistClass(dist) for dist in list(spjdf.Dist_To_Crash)]
                spjdf = spjdf.sort_values(['Dist_Class','F_SYSTEM','Dist_To_Crash'])
                #if list(spjdf.Dist_To_Crash)[0]<100:
                ext_rid = list(spjdf.ROUTE_ID)[0]
        if ext_rid!='':
            rdf = Route_Dict[year][Route_Dict[year]['ROUTE_ID']==ext_rid]
            pnt,disalongr,offset,side = rdf['Shape'].item().queryPointAndDistance(r['Shape'])
            Crash_Dict[year].set_value(i,'RID',ext_rid)
            Crash_Dict[year].set_value(i,'MP',pnt.firstPoint.M)
            Crash_Dict[year].set_value(i,'Offset',{True:1,False:-1}[side]*offset)#*3.28084)
        f.value += 1
print(strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
# Update Crash Data with RID and MP and Offset
print(strftime("%Y-%m-%d %H:%M:%S"))
for year in range(2012,2018):
    f = FloatProgress(min=0, max=int(str(arcpy.GetCount_management(CrashData[year]))),description = os.path.basename(CrashData[year]))
    display(f)
    arcpy.management.AddField(CrashData[year],'RID','Text',100)
    arcpy.management.AddField(CrashData[year],'MP','Double')
    arcpy.management.AddField(CrashData[year],'Offset','Double')
    uc = arcpy.UpdateCursor(CrashData[year])
    for r in uc:
        ucr = r.getValue('UCRnum')
        cdf = Crash_Dict[year][Crash_Dict[year]['UCRnum'] == ucr]
        r.setValue('RID'   ,cdf['RID'].item())
        r.setValue('MP'    ,cdf['MP'].item())
        r.setValue('Offset',cdf['Offset'].item())        
        uc.updateRow(r)
        f.value += 1 
    del uc
    del r
print(strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
# Map M crashes to P Direction
df = hsmpy3.common.FCtoDF(Routes[2016],selectedFields=['ROUTE_ID','Shape'])
Route_Dict = {year:df for year in range(2012,2018)}
Segs_Dict = {year:hsmpy3.common.FCtoDF(Segs[year],selectedFields=['ROUTE_ID','BEG_POINT','END_POINT','SegID']) for year in Years}
Segs_Dict.update({2017:Segs_Dict[2016]})
for year in range(2012,2018):
    if not 'RID_P' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'RID_P','Text',50)
    if not 'SegID_P' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'SegID_P','Text',50)
    if not 'MP_P' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'MP_P','Double')
    uc = arcpy.UpdateCursor(CrashData[year])
    for r in uc:
        if r.getValue('RID').split('-')[2] == 'M':
            rdf = Route_Dict[year][Route_Dict[year]['ROUTE_ID']==hsmpy3.nm.OppositeDirID(r.getValue('RID'))]
            pnt,disalongr,offset,side = rdf['Shape'].item().queryPointAndDistance(r.getValue('Shape'))
            mp = pnt.firstPoint.M
            sdf = Segs_Dict[year][Segs_Dict[year]['ROUTE_ID']==hsmpy3.nm.OppositeDirID(r.getValue('RID'))]
            if mp == max(list(sdf.END_POINT)):
                sdf = sdf[(sdf.BEG_POINT<=mp) & (sdf.END_POINT>=mp)]
            else:
                sdf = sdf[(sdf.BEG_POINT<=mp) & (sdf.END_POINT>mp)]
            SegID_P = sdf['SegID'].item()
            r.setValue('RID_P',str(hsmpy3.nm.OppositeDirID(r.getValue('RID'))))
            r.setValue('SegID_P',str(SegID_P))
            r.setValue('MP_P',float(mp))
            uc.updateRow(r)
    del uc
    del r
    print(year)

In [3]:
def DefineUrban(urban_code):
    if urban_code in [99999]:
        return(1)
    elif urban_code in [1171,27253,29278,47935,51499,79363,99998]:
        return(2)
    else:
        return(9)
def DefineFC(f_system):
    if not f_system in [1,2,3,4,5,6,7]:
        return(99)
    else:
        return(f_system)
FUN_SYS_Dict = {
        1: '01.Interstate',
        2: '02.Principal Arterial – Other Freeways and Expressways',
        3: '03.Principal Arterial – Other ',
        4: '04.Minor Arterial',
        5: '05.Major Collector',
        6: '06.Minor Collector',
        7: '07.Local',
        96: '08.Trafficway Not in State Inventory',
        98: '09.Not Reported',
        99: '10.Unknown'
}
RUR_URB_Dict = {
    1: '1.Rural',
    2: '2.Urban',
    6: '3.Trafficway Not in State Inventory',
    8: '4.Not Reported',
    9: '5.Unknown'
}
Crash_DF = pd.DataFrame()
for year in range(2012,2017):
    df = hsmpy3.common.FCtoDF(CrashData[year])
    df['Year'] = year
    Crash_DF = pd.concat([Crash_DF,df])
Crash_DF['FUN_SYS']    = [DefineFC(i) for  i in Crash_DF.F_SYSTEM]
Crash_DF['RUR_URB'] = [DefineUrban(i) for i in Crash_DF.URBAN_CODE]
Crash_DF['FC']    = [FUN_SYS_Dict[i] for  i in Crash_DF.FUN_SYS]
Crash_DF['RuralUrban'] = [RUR_URB_Dict[i] for i in Crash_DF.RUR_URB]

In [4]:
def DefineFCUR(FUN_SYS,RUR_URB):
    if FUN_SYS in [1] and RUR_URB in [2]:
        return('1.Urban - Interstate')
    if FUN_SYS in [1] and RUR_URB in [1]:
        return('2.Rural - Interstate')
    if FUN_SYS in [2]:
        return('3.PA - Other Freeway and Expressway')
    if FUN_SYS in [3] and RUR_URB in [1]:
        return('4.PA - Other Freeway and Expressway')
    if FUN_SYS in [3] and RUR_URB in [1]:
        return('5.PA - Other Freeway and Expressway')


In [21]:
def CrashAssignmentRes(rid,ft):
    FTypes = ['R4F','U4F','U6F','R2U','U2U','R4U','R4D','U4U','U4D','U6U']
    IFTypes = ['R3ST','R4ST','R4SG','RM3ST','RM4ST','RM4SG','U3ST','U4ST','U3SG','U4SG']
    if rid!='':
        if ft in FTypes:
            return('Assigned to Selected Roadways Types')
        elif ft in IFTypes:
            return('Assigned to Selected Intersection Types')
        else:
            return('Assigned to other FTypes')
    else:
        return('Not Assigned to Any Route')
Crash_DF['Assigned1'] = [CrashAssignmentRes(rid,ft) for rid,ft in zip(Crash_DF.RID,Crash_DF.FType)]

df = pd.DataFrame(Crash_DF.groupby(['Year','Assigned1'])['UCRnum'].aggregate(np.count_nonzero))
df['Year'] = [i[0] for i in list(df.index)]
df['Percentage'] = [d/float(sum(df[df.Year==y].UCRnum)) for d,y in zip(df.UCRnum,df.Year)]
df = df[['UCRnum','Percentage']]
df.columns = ['Count','Percentage']
df

Count  Percentage
Year Assigned1                                                 
2012 Assigned to Selected Intersection Types   1594        0.38
     Assigned to Selected Roadways Types        887        0.21
     Assigned to other FTypes                   930        0.22
     Not Assigned to Any Route                  732        0.18
2013 Assigned to Selected Intersection Types   1870        0.46
     Assigned to Selected Roadways Types        853        0.21
     Assigned to other FTypes                   511        0.13
     Not Assigned to Any Route                  835        0.21
2014 Assigned to Selected Intersection Types   1919        0.44
     Assigned to Selected Roadways Types        806        0.18
     Assigned to other FTypes                  1034        0.24
     Not Assigned to Any Route                  639        0.15
2015 Assigned to Selected Intersection Types   2401        0.49
     Assigned to Selected Roadways Types        890        0.18
     Assigned to other FTypes                  1119        0.23
     Not Assigned to Any Route                  493        0.10
2016 Assigned to Selected Intersection Types   2306        0.46
     Assigned to Selected Roadways Types        926        0.18
     Assigned to other FTypes                  1807        0.36
     Not Assigned to Any Route                    9        0.00

In [23]:
pd.options.display.float_format = "{:0.3f}".format
display(df[['Percentage']].unstack())
display(df[['Count']].unstack())


Percentage  \
Assigned1 Assigned to Selected Intersection Types   
Year                                                
2012                                        0.385   
2013                                        0.460   
2014                                        0.436   
2015                                        0.490   
2016                                        0.457   

                                                                        \
Assigned1 Assigned to Selected Roadways Types Assigned to other FTypes   
Year                                                                     
2012                                    0.214                    0.224   
2013                                    0.210                    0.126   
2014                                    0.183                    0.235   
2015                                    0.182                    0.228   
2016                                    0.183                    0.358   

                                     
Assigned1 Not Assigned to Any Route  
Year                                 
2012                          0.177  
2013                          0.205  
2014                          0.145  
2015                          0.101  
2016                          0.002

Count  \
Assigned1 Assigned to Selected Intersection Types   
Year                                                
2012                                         1594   
2013                                         1870   
2014                                         1919   
2015                                         2401   
2016                                         2306   

                                                                        \
Assigned1 Assigned to Selected Roadways Types Assigned to other FTypes   
Year                                                                     
2012                                      887                      930   
2013                                      853                      511   
2014                                      806                     1034   
2015                                      890                     1119   
2016                                      926                     1807   

                                     
Assigned1 Not Assigned to Any Route  
Year                                 
2012                            732  
2013                            835  
2014                            639  
2015                            493  
2016                              9

In [5]:
def ConvertROAD_FUNtoFUNC_SYS(road_fun):
    if road_fun<10:
        rur_urb = 1
    elif road_fun>10 and road_fun<20:
        rur_urb = 2
    else:
        rur_urb = 9
    fuc_conv_dict = {1:[1,11],2:[12],3:[2,13],4:[3,14],5:[4,15],6:[5],7:[6,16],96:[],98:[],99:[9,19,99]}
    fun_sys = [i for i in fuc_conv_dict if road_fun in fuc_conv_dict[i]][0]
    return([rur_urb,fun_sys])
State_Dict = {
    1:"Alabama",
    2:"Alaska",
    4:"Arizona",
    5:"Arkansas",
    6:"California",
    8:"Colorado",
    9:"Connecticut",
    10:"Delaware",
    11:"District of Columbia",
    12:"Florida",
    13:"Georgia",
    15:"Hawaii",
    16:"Idaho",
    17:"Illinois",
    18:"Indiana",
    19:"Iowa",
    20:"Kansas",
    21:"Kentucky",
    22:"Louisiana",
    23:"Maine",
    24:"Maryland",
    25:"Massachusetts",
    26:"Michigan",
    27:"Minnesota",
    28:"Mississippi",
    29:"Missouri",
    30:"Montana",
    31:"Nebraska",
    32:"Nevada",
    33:"New Hampshire",
    34:"New Jersey",
    35:"New Mexico",
    36:"New York",
    37:"North Carolina",
    38:"North Dakota",
    39:"Ohio",
    40:"Oklahoma",
    41:"Oregon",
    42:"Pennsylvania",
    43:"Puerto Rico",
    44:"Rhode Island",
    45:"South Carolina",
    46:"South Dakota",
    47:"Tennessee",
    48:"Texas",
    49:"Utah",
    50:"Vermont",
    51:"Virginia",
    52:"Virgin Islands",
    53:"Washington",
    54:"West Virginia",
    55:"Wisconsin",
    56:"Wyoming",
}
ROAD_FUN_Dict = {
        1: 'Rural-Principal Arterial – Interstate',
        2: 'Rural-Principal Arterial – Other',
        3: 'Rural-Minor Arterial',
        4: 'Rural-Major Collector',
        5: 'Rural-Minor Collector',
        6: 'Rural-Local Road or Street',
        9: 'Rural-Unknown Rural',
        11: 'Urban-Principal Arterial – Interstate',
        12: 'Urban-Principal Arterial – Other (Freeways or Expressways)',
        13: 'Urban-Other Principal Arterial',
        14: 'Urban-Minor Arterial',
        15: 'Urban-Collector',
        16: 'Urban-Local Road or Street',
        19: 'Urban-Unknown Urban',
        99: 'Unknown',
}
FARSDir = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO19\FARS Browser\1.Downloaded'

FARS_DF = pd.DataFrame()
for year in Years:
    df = pd.read_csv(FARSDir + '\\accident{}.csv'.format(year))
    df['Year'] = year
    if year<2015:
        df['FUNC_SYS'] = [ConvertROAD_FUNtoFUNC_SYS(i)[1] for i in list(df.ROAD_FNC)]
        df['RUR_URB' ] = [ConvertROAD_FUNtoFUNC_SYS(i)[0] for i in list(df.ROAD_FNC)]
    df['FC'] = [FUN_SYS_Dict[i] for i in list(df.FUNC_SYS)]
    df['RuralUrban'] = [RUR_URB_Dict[i] for i in list(df.RUR_URB)]
    FARS_DF = pd.concat([FARS_DF,df])
State_DF = FARS_DF[FARS_DF.STATE=={State_Dict[k]:k for k in State_Dict}['New Mexico']]
FARS_Occ_DF = pd.DataFrame()
for year in Years:
    df = pd.read_csv(FARSDir + '\\person{}.csv'.format(year))
    df['Year'] = year
    if year<2015:
        df['FUNC_SYS'] = [ConvertROAD_FUNtoFUNC_SYS(i)[1] for i in list(df.ROAD_FNC)]
        df['RUR_URB' ] = [ConvertROAD_FUNtoFUNC_SYS(i)[0] for i in list(df.ROAD_FNC)]
    df['FC'] = [FUN_SYS_Dict[i] for i in list(df.FUNC_SYS)]
    df['RuralUrban'] = [RUR_URB_Dict[i] for i in list(df.RUR_URB)]
    FARS_Occ_DF = pd.concat([FARS_Occ_DF,df])
FARS_Occ_DF['FARS_Ped_K'] = [{True:1,False:0}[v==4] for v in FARS_Occ_DF.INJ_SEV]
State_Occ_DF = FARS_Occ_DF[FARS_Occ_DF.STATE=={State_Dict[k]:k for k in State_Dict}['New Mexico']]
State_Ped_DF = State_Occ_DF[State_Occ_DF.PER_TYP==5]

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (19,20,21,22,29,30,32,33,36,37,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
Rows = ['Year']
Rows = ['FC']
#Rows = ['RuralUrban']
#Rows = ['Year','FC','RuralUrban']
#Rows = ['Year','FC','RuralUrban']
Res_FARS = pd.DataFrame(State_DF.groupby(Rows)['FATALS'].aggregate(np.sum))
Res_State = pd.DataFrame(Crash_DF.groupby(Rows)['Killed', 'ClassA', 'ClassB'].aggregate(np.sum))

Res_Comb = pd.concat([Res_State, Res_FARS], axis=1)
Res_Comb = Res_Comb.fillna(0)
Res_Comb = Res_Comb[['FATALS','Killed','ClassA','ClassB']]
Res_Comb.columns = ['FARS_Killed','State_Killed','State_ClassA','State_ClassB']
Res_Comb = Res_Comb.sort_index(level=range(0,len(Rows)))
Res_Comb = Res_Comb.astype(int)
Res_Comb.to_excel('Crash_Assignment.xlsx')
display(HTML(Res_Comb.to_html()))

,FARS_Killed,State_Killed,State_ClassA,State_ClassB
01.Interstate,415,411,1004,3044
02.Principal Arterial – Other Freeways and Expressways,29,5,20,51
03.Principal Arterial – Other,610,674,2985,8495
04.Minor Arterial,193,239,1150,3545
05.Major Collector,154,233,682,2334
06.Minor Collector,54,53,137,505
07.Local,205,128,544,1936
08.Trafficway Not in State Inventory,7,0,0,0
09.Not Reported,7,0,0,0
10.Unknown,89,17,79,181


In [7]:
print(sum(Res_Comb.State_Killed))
print(sum(Res_Comb.FARS_Killed))

1760
1763


In [31]:
Ped_Res_State = pd.DataFrame(Crash_DF.groupby(['Year'])['PedK','PedA','PedB'].aggregate(np.sum))
Ped_Res_FARS  = pd.DataFrame(State_Ped_DF.groupby(['Year'])['FARS_Ped_K'].aggregate(np.sum))
Ped_Res_Comb = pd.concat([Ped_Res_FARS,Ped_Res_State],axis=1)
Ped_Res_Comb.columns = ['FARS_Ped_Killed','State_Ped_Killed','State_Ped_ClassA','State_Ped_ClassB']
Ped_Res_Comb = Ped_Res_Comb.astype(int)
Ped_Res_Comb

,FARS_Ped_Killed,State_Ped_Killed,State_Ped_ClassA,State_Ped_ClassB
Year,,,,
2012,75,61,58,129
2013,49,53,95,141
2014,75,74,94,189
2015,54,55,126,211
2016,73,77,84,204


In [34]:
Ped_Res_State = pd.DataFrame(Crash_DF[Crash_DF.Year<2017].groupby(['FC'])['PedK','PedA','PedB'].aggregate(np.sum))
Ped_Res_FARS  = pd.DataFrame(State_Ped_DF.groupby(['FC'])['FARS_Ped_K'].aggregate(np.sum))
Ped_Res_Comb = pd.concat([Ped_Res_FARS,Ped_Res_State],axis=1)
Ped_Res_Comb.columns = ['FARS_Ped_Killed','State_Ped_Killed','State_Ped_ClassA','State_Ped_ClassB']
Ped_Res_Comb.columns.name = 'Functional Class'
Ped_Res_Comb = Ped_Res_Comb.fillna(0)
Ped_Res_Comb = Ped_Res_Comb.astype(int)
Ped_Res_Comb

Functional Class,FARS_Ped_Killed,State_Ped_Killed,State_Ped_ClassA,State_Ped_ClassB
01.Interstate,72,51,20,9
02.Principal Arterial – Other Freeways and Expressways,7,0,0,0
03.Principal Arterial – Other,151,135,200,347
04.Minor Arterial,39,40,66,165
05.Major Collector,17,27,26,69
06.Minor Collector,3,13,38,81
07.Local,27,25,87,175
08.Trafficway Not in State Inventory,2,0,0,0
10.Unknown,8,29,20,28


In [22]:
OwnerShipDesc = {
    0:'Not Assigned',
    1:'State Highway Agency',
    2:'County Highway Agency',
    3:'Town or Township Highway Agency',
    4:'City or Municipal Highway Agency',
    60:'Other Federal Agency',
    62:'Bureau of Indian Affairs',
    64:'U.S. Forest Service',
}
def FindO(o):
    if o in list(OwnerShipDesc.keys()):
        return(OwnerShipDesc[o])
    else:
        return('Unknown')
Crash_DF['KA'] = Crash_DF.Killed + Crash_DF.ClassA
Crash_DF['Ownership'] = [FindO(o) for o in Crash_DF.OWNERSHIP]
df = pd.DataFrame(Crash_DF.groupby(['Year','FC','Ownership'])['Killed', 'ClassA', 'KA'].aggregate(np.sum))
df = df.unstack('Year')
df.to_excel('OwnershipandFC.xlsx')